# 01 Import libraries

In [2]:
import numpy as np
import torch

device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using device: {device}")

Using device: cpu


In [3]:
import pandas as pd

In [4]:
%reload_ext autoreload
%autoreload 2

In [5]:
import importlib

In [6]:
import lib
importlib.reload(lib)

<module 'lib' from 'c:\\Users\\dongq\\OneDrive\\Desktop\\New results\\Portfolio Allocation\\Learning-the-Optimal-Solution-Path\\lib\\__init__.py'>

In [23]:
from lib.lsp.basis_generator import dampen_laguerre
from lib.lsp.basis_tf_module import Basis_TF_SGD
from lib.lsp.learn_solution_path import learn_solution_path
from lib.lsp.loss_fn_lsp import reg_exp_weighted_logit
from lib.fast_tensor_data_loader import FastTensorDataLoader
from lib.lsp.utils_lsp import get_errs_lsp

# 02 Instantiate dataset

In [8]:
# file path
X_df = pd.read_csv('X_processed.csv')
y_df = pd.read_csv('y_processed.csv')

In [9]:
X = np.array(X_df)
y = np.array(y_df).squeeze()

In [10]:
train_X = torch.tensor(X, dtype=torch.float32)
train_y = torch.tensor(y, dtype=torch.float32)

In [11]:
# full gradient descent uses all data points
GD_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=True, )
# test data
test_data_loader = FastTensorDataLoader(train_X, train_y, batch_size=1000, shuffle=False, )

In [12]:
lam_max = [20]
lam_min = [0]
input_dim = X.shape[1]
loss_fn = reg_exp_weighted_logit

In [13]:
# Read the CSV file into a DataFrame
truth = pd.read_csv('exact_soln_list_laguerre.csv')

# Display the DataFrame
truth

,losses,theta_0,theta_1,theta_2,theta_3,theta_4,theta_5,theta_6,theta_7,theta_8,...,theta_36,theta_37,theta_38,theta_39,theta_40,theta_41,theta_42,theta_43,theta_44,theta_45
0,0.251592,-0.290164,0.186074,0.232884,-0.027476,0.166136,0.110354,0.142358,0.009975,0.157656,...,0.000471,0.023097,-0.007038,0.003151,0.004510,-0.024440,0.000471,-0.129229,-0.096597,-0.041242
1,0.251645,-0.290092,0.186073,0.232887,-0.027478,0.166136,0.110326,0.142336,0.009961,0.157669,...,0.000487,0.023069,-0.007039,0.003154,0.004514,-0.024434,0.000487,-0.129220,-0.096579,-0.041224
2,0.251698,-0.290020,0.186071,0.232891,-0.027481,0.166136,0.110298,0.142315,0.009946,0.157682,...,0.000503,0.023042,-0.007041,0.003157,0.004518,-0.024429,0.000503,-0.129210,-0.096561,-0.041206
3,0.251751,-0.289947,0.186070,0.232895,-0.027484,0.166136,0.110270,0.142294,0.009931,0.157695,...,0.000519,0.023015,-0.007042,0.003160,0.004522,-0.024423,0.000519,-0.129201,-0.096543,-0.041188
4,0.251804,-0.289874,0.186068,0.232898,-0.027487,0.166136,0.110242,0.142272,0.009916,0.157708,...,0.000534,0.022988,-0.007043,0.003162,0.004526,-0.024417,0.000534,-0.129191,-0.096525,-0.041170
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1019,0.425709,0.608973,0.015982,0.030750,-0.012680,0.033522,-0.189898,-0.060873,-0.075246,0.138251,...,0.104496,-0.190631,-0.003899,0.016846,0.022910,0.049139,0.104496,0.117941,0.139251,0.161150
1020,0.405809,0.627298,0.006731,0.017247,-0.013635,0.025534,-0.197376,-0.075369,-0.070611,0.125166,...,0.104706,-0.193085,-0.002800,0.016029,0.021890,0.052167,0.104706,0.126507,0.144123,0.163584
1021,0.382318,0.647696,-0.004087,0.001661,-0.014793,0.016128,-0.205935,-0.094402,-0.064959,0.109066,...,0.104903,-0.195830,-0.001589,0.015089,0.020769,0.055620,0.104903,0.136154,0.149574,0.166138
1022,0.354260,0.670989,-0.016946,-0.016535,-0.016181,0.004871,-0.216011,-0.119855,-0.058109,0.089182,...,0.105111,-0.199002,-0.000252,0.014026,0.019573,0.059569,0.105111,0.147241,0.155866,0.168880


In [14]:
selected_columns = ['theta_0', 'theta_1', 'theta_2', 'theta_3', 'theta_4',
                    'theta_5', 'theta_6', 'theta_7', 'theta_8', 'theta_9',
                    'theta_10', 'theta_11', 'theta_12', 'theta_13', 'theta_14',
                    'theta_15', 'theta_16', 'theta_17', 'theta_18', 'theta_19',
                    'theta_20', 'theta_21', 'theta_22', 'theta_23', 'theta_24',
                    'theta_25', 'theta_26', 'theta_27', 'theta_28', 'theta_29',
                    'theta_30', 'theta_31', 'theta_32', 'theta_33', 'theta_34',
                    'theta_35', 'theta_36', 'theta_37', 'theta_38', 'theta_39',
                    'theta_40', 'theta_41', 'theta_42', 'theta_43', 'theta_44',
                    'theta_45']
true_thetas = truth[selected_columns].to_numpy()
true_losses = truth['losses'].to_numpy()

# 03 SGD with Diminishing LR by Distance Diagnostic

Recall that our method runs SGD over random $\tilde λ$'s with a linear basis $\Phi(\tilde \lambda)$ of our choice. We want to approximate $\theta$ with $\Phi(\lambda)\beta$, so the objective function is $\min_\beta h(\Phi(\tilde\lambda)\beta, \tilde\lambda) = (1-\tilde\lambda) BCE(X_\text{pass}\Phi(\tilde\lambda)\beta,\ y_\text{pass}) + \tilde\lambda BCE(X_\text{fail}\Phi(\tilde\lambda)\beta,\ y_\text{fail})$. For each batch of training data set, we randomize $\tilde\lambda$. If batch size = 1, then this is equivalent to a standard SGD.

We use diminishing learning rate for better demonstrate convergence. If we use a constant learning rate, the solution path error will eventually do a random walk after descending to a certain threshold value.

Set weighted_avg to 'False' see this random walk.

In [15]:
max_epochs = 6000

In [16]:
phi_lam = dampen_laguerre

In [17]:
def thresh_basis(basis_dim):
    return 1e-9

## num basis func = 5

In [52]:
basis_dim = 5
init_lr = 0.0625

In [54]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history_last_itr, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               weighted_avg=False,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=100)

sup_err_history_last_itr = np.array(sup_err_history_last_itr)

--------approximate solution path for # itr = 100 complete--------
# epoch: 100	 sup error: 0.1301219761371613
--------approximate solution path for # itr = 200 complete--------
# epoch: 200	 sup error: 0.0748749971389771
--------approximate solution path for # itr = 300 complete--------
# epoch: 300	 sup error: 0.057764887809753473
--------approximate solution path for # itr = 400 complete--------
# epoch: 400	 sup error: 0.14746946096420294
--------approximate solution path for # itr = 500 complete--------
# epoch: 500	 sup error: 0.28371706604957586
--------approximate solution path for # itr = 600 complete--------
# epoch: 600	 sup error: 0.14867219328880316
--------approximate solution path for # itr = 700 complete--------
# epoch: 700	 sup error: 0.2867776453495026
--------approximate solution path for # itr = 800 complete--------
# epoch: 800	 sup error: 0.3386392295360566
--------approximate solution path for # itr = 900 complete--------
# epoch: 900	 sup error: 0.0678423345088

In [55]:
file_path = 'LSP_results_exact_fixed_basis_laguerre_last_iterate_shrink_lr.csv'

LSP_results_exact_fixed_basis = pd.DataFrame(np.column_stack((num_itr_history, sup_err_history_last_itr)), columns=['num_itr', 'sup_err_5'])

# Save the DataFrame to a CSV file
LSP_results_exact_fixed_basis.to_csv(file_path, index=False)

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

# Display the DataFrame
df

,num_itr,sup_err_5
0,10.0,0.110583
1,20.0,0.147659
2,30.0,0.437088
3,40.0,0.419268
4,50.0,0.064437
...,...,...
595,5960.0,0.194845
596,5970.0,0.204207
597,5980.0,0.217541
598,5990.0,0.265227


## num basis func = 7

In [56]:
basis_dim = 7
init_lr = 0.0625

In [57]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history_last_itr, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               weighted_avg=False,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=100)

sup_err_history_last_itr = np.array(sup_err_history_last_itr)

--------approximate solution path for # itr = 100 complete--------
# epoch: 100	 sup error: 0.05680218338966375
--------approximate solution path for # itr = 200 complete--------
# epoch: 200	 sup error: 0.014192730188369806
--------approximate solution path for # itr = 300 complete--------
# epoch: 300	 sup error: 0.01246184110641485
--------approximate solution path for # itr = 400 complete--------
# epoch: 400	 sup error: 0.033269077539444025
--------approximate solution path for # itr = 500 complete--------
# epoch: 500	 sup error: 0.0693494081497193
--------approximate solution path for # itr = 600 complete--------
# epoch: 600	 sup error: 0.04959860444068914
--------approximate solution path for # itr = 700 complete--------
# epoch: 700	 sup error: 0.06317484378814703
--------approximate solution path for # itr = 800 complete--------
# epoch: 800	 sup error: 0.08454054594039923
--------approximate solution path for # itr = 900 complete--------
# epoch: 900	 sup error: 0.022322952

In [58]:
file_path = 'LSP_results_exact_fixed_basis_laguerre_last_iterate_shrink_lr.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_7'] = sup_err_history_last_itr

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

## num basis func = 9

In [18]:
basis_dim = 9
init_lr = 0.0625

In [20]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history_last_itr, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               weighted_avg=False,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=1000)

sup_err_history_last_itr = np.array(sup_err_history_last_itr)

--------approximate solution path for # itr = 1000 complete--------
# epoch: 1000	 sup error: 0.013005137443542536
--------approximate solution path for # itr = 2000 complete--------
# epoch: 2000	 sup error: 0.01155266165733343
--------approximate solution path for # itr = 3000 complete--------
# epoch: 3000	 sup error: 0.021070331335067805
--------approximate solution path for # itr = 4000 complete--------
# epoch: 4000	 sup error: 0.020676285028457697
--------approximate solution path for # itr = 5000 complete--------
# epoch: 5000	 sup error: 0.022442072629928644
--------approximate solution path for # itr = 6000 complete--------
# epoch: 6000	 sup error: 0.01965263485908514


In [60]:
np.random.seed(8675309)
torch.manual_seed(8675309)

num_itr_history, sup_err_history_last_itr, weight, lr, itr = learn_solution_path(input_dim, basis_dim, phi_lam, max_epochs,
                                                               GD_data_loader, test_data_loader, loss_fn,
                                                               lam_min, lam_max, true_losses, init_lr=init_lr,
                                                               diminish=True, q=1.5, thresh_basis=thresh_basis,
                                                               weighted_avg=False,
                                                               record_frequency=10, distribution='exponential',
                                                               device=device, trace_frequency=100)

sup_err_history_last_itr = np.array(sup_err_history_last_itr)

--------approximate solution path for # itr = 100 complete--------
# epoch: 100	 sup error: 0.04896950721740728
--------approximate solution path for # itr = 200 complete--------
# epoch: 200	 sup error: 0.007388442754745539
--------approximate solution path for # itr = 300 complete--------
# epoch: 300	 sup error: 0.012091338634490967
--------approximate solution path for # itr = 400 complete--------
# epoch: 400	 sup error: 0.01395034790039068
--------approximate solution path for # itr = 500 complete--------
# epoch: 500	 sup error: 0.017959564924240168
--------approximate solution path for # itr = 600 complete--------
# epoch: 600	 sup error: 0.01897063851356512
--------approximate solution path for # itr = 700 complete--------
# epoch: 700	 sup error: 0.022045165300369318
--------approximate solution path for # itr = 800 complete--------
# epoch: 800	 sup error: 0.02689382433891302
--------approximate solution path for # itr = 900 complete--------
# epoch: 900	 sup error: 0.013307

In [61]:
file_path = 'LSP_results_exact_fixed_basis_laguerre_last_iterate_shrink_lr.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df['sup_err_9'] = sup_err_history_last_itr

# Save the DataFrame to a CSV file
df.to_csv(file_path, index=False)

# 04 Error Across $\Lambda$

In [21]:
model = Basis_TF_SGD(input_dim, 9, phi_lam, init_weight=weight, intercept=True).to(device)

In [24]:
errs = get_errs_lsp(lam_min[0], lam_max[0], true_losses, model, test_data_loader, loss_fn)

In [26]:
lambdas = np.linspace(lam_max[0], lam_min[0], len(true_losses))

In [28]:
pd.DataFrame(np.column_stack((errs, lambdas)), columns=['errs', 'lambdas']).to_csv('LSP_errs_laguerre_basis_9.csv', index=False)